# Renovation Graph Node Testing

This notebook allows you to test individual nodes of the renovation estimation pipeline.

## Modes

- **Fixture mode** (`MODE = "fixture"`): Uses offline fixture data, no API calls
- **Live mode** (`MODE = "live"`): Makes real API calls to Apify/OpenAI

## Pipeline Steps

1. **Scrape**: Fetch property data from Idealista via Apify
2. **Classify**: Classify each image to identify room types
3. **Group**: Group images by room (pure logic, always runs live)
4. **Estimate**: Analyze each room and estimate renovation costs
5. **Summarize**: Generate final report with totals

## 1. Setup

In [ ]:
# Add parent directory to path so we can import app modules
import sys
from pathlib import Path

backend_dir = Path.cwd().parent
if str(backend_dir) not in sys.path:
    sys.path.insert(0, str(backend_dir))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(backend_dir / ".env")

# Imports
import json
from pprint import pprint
from app.config import Settings
from app.graphs.main_graph import (
    scrape_node,
    classify_node,
    group_node,
    estimate_node,
    summarize_node,
    build_renovation_graph,
)
from app.graphs.state import create_initial_state
from fixtures import get_state_after, SAMPLE_URL

print("✅ Setup complete")

## 2. Mode Selection

In [ ]:
# Choose mode: "fixture" or "live"
MODE = "fixture"  # Change to "live" for real API calls

# Initialize settings
settings = Settings()

# For live mode, optionally override the sample URL
IDEALISTA_URL = SAMPLE_URL  # Change this for live mode

print(f"Mode: {MODE}")
print(f"URL: {IDEALISTA_URL}")
if MODE == "live":
    print(f"OpenAI Model (estimation): {settings.openai_vision_model}")
    print(f"OpenAI Model (classification): {settings.openai_classification_model}")

## 3. Load Fixtures

In [ ]:
# Show available fixture stages
stages = ["initial", "scrape", "classify", "group", "estimate", "summarize"]
print("Available fixture stages:")
for stage in stages:
    state = get_state_after(stage)
    print(f"  - {stage}: current_step='{state.get('current_step', 'N/A')}'")

## 4. Helper Functions

In [ ]:
def show_state(state, keys=None):
    """Pretty-print specific keys from state."""
    if keys is None:
        keys = ["current_step", "error"]
    
    subset = {k: state.get(k) for k in keys if k in state}
    pprint(subset)


def show_events(state):
    """Show stream events from state."""
    events = state.get("stream_events", [])
    print(f"\nStream Events ({len(events)}):")
    for i, event in enumerate(events, 1):
        msg = event.message if hasattr(event, 'message') else event.get('message', '')
        event_type = event.type if hasattr(event, 'type') else event.get('type', '')
        print(f"  {i}. [{event_type}] {msg}")


def show_property_data(state):
    """Show scraped property data."""
    prop = state.get("property_data")
    if prop:
        print(f"\nProperty Data:")
        print(f"  Title: {prop.title if hasattr(prop, 'title') else prop.get('title', 'N/A')}")
        print(f"  Price: {prop.price if hasattr(prop, 'price') else prop.get('price', 0):,.0f}€")
        print(f"  Area: {prop.area_m2 if hasattr(prop, 'area_m2') else prop.get('area_m2', 0)} m²")
        num_images = len(prop.image_urls) if hasattr(prop, 'image_urls') else len(prop.get('image_urls', []))
        print(f"  Images: {num_images}")


def show_classifications(state):
    """Show classification results."""
    classifications = state.get("classifications", [])
    print(f"\nClassifications ({len(classifications)}):")
    for i, c in enumerate(classifications, 1):
        room_type = c.room_type.value if hasattr(c, 'room_type') else c.get('room_type', 'N/A')
        room_num = c.room_number if hasattr(c, 'room_number') else c.get('room_number', 0)
        conf = c.confidence if hasattr(c, 'confidence') else c.get('confidence', 0)
        print(f"  {i}. {room_type} #{room_num} (confidence: {conf:.2f})")


def show_grouped_images(state):
    """Show grouped images by room."""
    grouped = state.get("grouped_images", {})
    print(f"\nGrouped Images ({len(grouped)} rooms):")
    for room_key, images in grouped.items():
        print(f"  {room_key}: {len(images)} image(s)")


def show_room_analyses(state):
    """Show room analysis results."""
    analyses = state.get("room_analyses", [])
    print(f"\nRoom Analyses ({len(analyses)}):")
    for analysis in analyses:
        label = analysis.room_label if hasattr(analysis, 'room_label') else analysis.get('room_label', 'N/A')
        condition = analysis.condition.value if hasattr(analysis, 'condition') else analysis.get('condition', 'N/A')
        cost_min = analysis.cost_min if hasattr(analysis, 'cost_min') else analysis.get('cost_min', 0)
        cost_max = analysis.cost_max if hasattr(analysis, 'cost_max') else analysis.get('cost_max', 0)
        num_items = len(analysis.renovation_items) if hasattr(analysis, 'renovation_items') else len(analysis.get('renovation_items', []))
        print(f"  {label}: {condition} | {cost_min:,.0f}€ - {cost_max:,.0f}€ | {num_items} items")


def show_estimate(state):
    """Show final estimate."""
    estimate = state.get("estimate")
    if estimate:
        total_min = estimate.total_cost_min if hasattr(estimate, 'total_cost_min') else estimate.get('total_cost_min', 0)
        total_max = estimate.total_cost_max if hasattr(estimate, 'total_cost_max') else estimate.get('total_cost_max', 0)
        summary = estimate.summary if hasattr(estimate, 'summary') else estimate.get('summary', '')
        print(f"\nFinal Estimate:")
        print(f"  Total: {total_min:,.0f}€ - {total_max:,.0f}€")
        print(f"  Summary: {summary[:150]}..." if len(summary) > 150 else f"  Summary: {summary}")

print("✅ Helper functions loaded")

## 5. Node 1: Scrape

Fetch property data from Idealista via Apify.

In [ ]:
if MODE == "fixture":
    # Load fixture state
    state_after_scrape = get_state_after("scrape")
    print("✅ Loaded fixture state after scrape")
else:
    # Run live scrape
    initial_state = create_initial_state(IDEALISTA_URL, user_id="notebook_test")
    state_after_scrape = await scrape_node(initial_state, settings=settings)
    print("✅ Scrape completed")

show_state(state_after_scrape, ["current_step", "error"])
show_property_data(state_after_scrape)
show_events(state_after_scrape)

## 6. Node 2: Classify

Classify each image to identify room types using GPT-4o-mini.

In [ ]:
if MODE == "fixture":
    # Load fixture state
    state_after_classify = get_state_after("classify")
    print("✅ Loaded fixture state after classify")
else:
    # Run live classify
    state_after_classify = await classify_node(state_after_scrape, settings=settings)
    print("✅ Classify completed")

show_state(state_after_classify, ["current_step", "error"])
show_classifications(state_after_classify)
show_events(state_after_classify)

## 7. Node 3: Group

Group images by room (pure logic, no API calls).

In [ ]:
# Always run live (pure logic, no API calls)
if MODE == "fixture":
    state_after_group = get_state_after("group")
    print("✅ Loaded fixture state after group")
else:
    state_after_group = await group_node(state_after_classify, settings=settings)
    print("✅ Group completed")

show_state(state_after_group, ["current_step", "error"])
show_grouped_images(state_after_group)
show_events(state_after_group)

## 8. Node 4: Estimate

Analyze each room and estimate renovation costs using GPT-4o Vision.

In [ ]:
if MODE == "fixture":
    # Load fixture state
    state_after_estimate = get_state_after("estimate")
    print("✅ Loaded fixture state after estimate")
else:
    # Run live estimate
    state_after_estimate = await estimate_node(state_after_group, settings=settings)
    print("✅ Estimate completed")

show_state(state_after_estimate, ["current_step", "error"])
show_room_analyses(state_after_estimate)
show_events(state_after_estimate)

## 9. Node 5: Summarize

Generate final report with totals and summary.

In [ ]:
if MODE == "fixture":
    # Load fixture state
    state_after_summarize = get_state_after("summarize")
    print("✅ Loaded fixture state after summarize")
else:
    # Run live summarize
    state_after_summarize = await summarize_node(state_after_estimate, settings=settings)
    print("✅ Summarize completed")

show_state(state_after_summarize, ["current_step", "error"])
show_estimate(state_after_summarize)
show_events(state_after_summarize)

## 10. Full Graph (Live Mode Only)

Run the complete graph end-to-end.

In [ ]:
if MODE == "live":
    print("Running full graph end-to-end...")
    graph = build_renovation_graph(settings)
    initial_state = create_initial_state(IDEALISTA_URL, user_id="notebook_test")
    
    final_state = await graph.ainvoke(initial_state)
    
    print("\n✅ Full graph completed")
    show_state(final_state, ["current_step", "error"])
    show_estimate(final_state)
else:
    print("⚠️  Full graph execution only available in live mode")

## 11. Graph Visualization

Display the graph structure.

In [ ]:
# Build graph and show structure
graph = build_renovation_graph(settings)

try:
    # Try to display as mermaid if available
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # Fallback to ASCII
    print("Graph structure:")
    print("""\nSCRAPE → CLASSIFY → GROUP → ESTIMATE → SUMMARIZE → END\n""")
    print("Each node receives state and settings as parameters.")